In [17]:
# -*- coding:utf-8 -*- ＃
import sys
import time
import networkx as nx
from operator import add
import numpy as np
def buildinginone(l):
    if u"图书馆" in l:
        return u"图书馆"
    if u"工程训练中心" in l:
        return u"工程训练中心"
    if u"新行政楼" in l:
        return u"新行政楼"
    if u"系统生物" in l:
        return u"系统生物"
    if u"航空航天" in l:
        return u"航空航天学院"
    if u"宿舍" in l:
        return "宿舍"
    return l

In [2]:
holiday=['2014-10-01', '2014-10-02', '2014-10-03','2014-10-04', '2014-10-05', '2014-10-06', '2014-10-07','2015-01-01','2015-01-02','2015-01-03']

In [18]:
day=sc.textFile("./place",100).map(lambda x:x.split(",")[0]).distinct().filter(lambda x:x not in holiday)
dl=sorted(day.collect())
data=sc.textFile("./place",100).map(lambda x:x.split(",",2)).map(lambda x:(buildinginone(x[1]),(x[0],x[2]))).groupByKey().mapValues(list)

In [47]:
def tuple2dic(l):
    dic={}
    for i in l:
        if i[0] not in holiday:
            dic[i[0]]=eval(i[1])+dic.get(i[0],[])
    for j in dl:
        l=dic.get(j,[])
        dic[j]=dic.get(j,[])
    return dic
alldaydata=data.mapValues(tuple2dic)

In [60]:
# count=alldaydata.map(lambda x:len(x[1])).distinct()
place=alldaydata.map(lambda x:x[0]).distinct().collect()
datedic={}
for i in dl:
    datedic[i]=str(dl.index(i))
placedic={}
for i in place:
    placedic[i]=str(place.index(i))

In [68]:
def extract_feature(l):
    length=len(l)
    one=0
    two=0
    three=0
    for i in l:
        if i<4:
            three+=1
        if i<3:
            two+=1
        if i<2:
            one+=1
    re=[length,one,two,three]
    re=map(str,re)
    return re
input=alldaydata.flatMapValues(lambda x:x.items())\
                .map(lambda x:((x[0],x[1][0]),x[1][1]))\
                .mapValues(extract_feature)\
                .map(lambda x:placedic.get(x[0][0])+","+datedic.get(x[0][1])+","+",".join(x[1]))

In [69]:
input.cache()

PythonRDD[101] at RDD at PythonRDD.scala:43

In [71]:
input.first()

'0,64,150,18,25,29'

In [18]:
l=[]
for i in dl:
    l.append(time.localtime(time.mktime(time.strptime(i,"%Y-%m-%d"))).tm_wday)

In [22]:
dl.sort()
dl

['2014-09-15',
 '2014-09-16',
 '2014-09-17',
 '2014-09-18',
 '2014-09-19',
 '2014-09-22',
 '2014-09-23',
 '2014-09-24',
 '2014-09-25',
 '2014-09-26',
 '2014-09-29',
 '2014-09-30',
 '2014-10-08',
 '2014-10-09',
 '2014-10-10',
 '2014-10-13',
 '2014-10-14',
 '2014-10-15',
 '2014-10-16',
 '2014-10-17',
 '2014-10-20',
 '2014-10-21',
 '2014-10-22',
 '2014-10-23',
 '2014-10-24',
 '2014-10-27',
 '2014-10-28',
 '2014-10-29',
 '2014-10-30',
 '2014-10-31',
 '2014-11-03',
 '2014-11-04',
 '2014-11-05',
 '2014-11-06',
 '2014-11-07',
 '2014-11-10',
 '2014-11-11',
 '2014-11-12',
 '2014-11-13',
 '2014-11-14',
 '2014-11-17',
 '2014-11-18',
 '2014-11-19',
 '2014-11-20',
 '2014-11-21',
 '2014-11-24',
 '2014-11-25',
 '2014-11-26',
 '2014-11-27',
 '2014-11-28',
 '2014-12-01',
 '2014-12-02',
 '2014-12-03',
 '2014-12-04',
 '2014-12-05',
 '2014-12-08',
 '2014-12-09',
 '2014-12-10',
 '2014-12-11',
 '2014-12-12',
 '2014-12-15',
 '2014-12-16',
 '2014-12-17',
 '2014-12-18',
 '2014-12-19',
 '2014-12-22',
 '2014-12-